In [ ]:
#!pip install umap-learn[plot]

In [ ]:
import os
import graphistry
import pandas as pd
#import torch.nn.functional as F

#from ml.dgl_utils import *
from ml.utils import *
from ml.feature_utils import *
from ml.umap_utils import *

import umap.plot

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#graphistry.register(api=3, protocol="https", server="hub.graphistry.com", username="..", password="..")
graphistry.register(api=3, protocol="https", server="hub.graphistry.com", username=os.environ['USERNAME'], password=os.environ['GRAPHISTRY_PASSWORD']) 

In [ ]:
src, dst = 'to_node', 'from_node'

In [ ]:
edf = pd.read_csv('data/edges_blackrock.csv', index_col=0)
ndf = pd.read_csv('data/nodes_blackrock.csv', index_col=0)
ndf = ndf.fillna('').astype(str)
edf = edf.drop(columns=['src', 'dst']).astype(str)

In [ ]:
edf

In [ ]:
ndf

# Featurize

In [ ]:
fc = FeatureMixin(**umap_kwargs_probs)

In [ ]:
g = fc.edges(edf, 'to_node', 'from_node').nodes(ndf, 'Node')

In [ ]:
g2 = g.featurize(kind='edges', y = pd.DataFrame({'relationship_type': edf.relationship_type.values}))

In [ ]:
g2 = g.featurize(kind='nodes', y = pd.DataFrame({'Types': ndf.Types.values}))

In [ ]:
N = g.node_target.values.argmax(1) # can make an effective label target for Nodes

In [ ]:
g.node_features

In [ ]:
g.edge_features

In [ ]:
E=g2.edge_target.values.argmax(1) # effective edge target

In [ ]:
g3 = g2.umap()

In [ ]:
g3.xy_nodes

In [ ]:
plt.figure(figsize=(10,10)); plt.scatter(g.xy_nodes.T[0], g.xy_nodes.T[1], c=N, s=80, alpha=0.4)

In [ ]:
g4 = g.umap(kind='edges', scale=1)

In [ ]:
plt.figure(figsize=(10,10)); plt.scatter(g.xy_edges.T[0], g.xy_edges.T[1], c=E, s=80, alpha=0.4)

In [ ]:
g.weighted_adjacency_edges

In [ ]:
g4.weighted_edges_df_from_edges 

In [ ]:
g.weighted_edges_df_from_nodes

In [ ]:
g.weighted_adjacency_edges

In [ ]:
X=g.node_features

In [ ]:
g4 = g.umap(kind='nodes', X=g.node_features, y = N)

In [ ]:
g4 = g.umap(kind='nodes', X=g.node_features, y = N.reshape(-1, 1))

In [ ]:
umap.plot.points(g, labels=N, theme='fire')

In [ ]:
g4 = g.umap(kind='nodes', X=g.node_features, y = None) # without the target influencing the UMAP

In [ ]:
umap.plot.points(g, labels=N, theme='fire')

In [ ]:
umap.plot.output_notebook()

In [ ]:
# need to reindex to get contiguous index  
hover_data = pd.DataFrame(g._nodes.values, index=range(len(g._nodes)), columns=g._nodes.columns)

In [ ]:
hover_data = hover_data.fillna('')

In [ ]:
hover_data = hover_data[['Node', 'Blurb', 'Summary','Types']]

In [ ]:
hover_data

In [ ]:
p = umap.plot.interactive(g, labels=N, hover_data=hover_data, point_size=20, theme='fire')
umap.plot.show(p)

# Now we can do the same for Edges

In [ ]:
hover_data2 = pd.DataFrame(g._edges.values, index=range(len(g._edges)), columns=g._edges.columns)

In [ ]:
g2=g.umap(kind='edges')

In [ ]:
p = umap.plot.interactive(g, labels=E, hover_data=hover_data2, point_size=20, theme='fire')
umap.plot.show(p)

In [ ]:
g.plot()

In [ ]:
#umap.plot.connectivity(g, edge_bundling='hammer')

In [ ]:
for mindist in np.linspace(0,1,5):
    g.umap(kind='nodes', **{'min_dist':mindist})
    p = umap.plot.interactive(g, labels=N, hover_data=hover_data, point_size=20, theme='fire')
    umap.plot.show(p)
    

In [ ]:
for k in range(2,62,10):
    g.umap(kind='nodes', **{'n_neighbors':k})
    print(k)
    p = umap.plot.interactive(g, labels=N, hover_data=hover_data, point_size=20, theme='fire')
    umap.plot.show(p)
    

In [ ]:
g.xy_nodes
# g2 = g.edges(edf, 'to_node', 'from_node').nodes(ndf, 'Node').bind(point_x='x', point_y='y')
# g2.plot()

In [ ]:
wdf = g.weighted_edges_df_from_nodes

In [ ]:
coldict = dict(zip(range(len(g._nodes)), g._nodes['Node'].values))

In [ ]:
wdf['src'] = wdf['_src'].apply(lambda x: coldict[x])
wdf['dst'] = wdf['_dst'].apply(lambda x: coldict[x])

In [ ]:
wdf

In [ ]:
hover_data['_x'] = 10*g.xy_nodes.T[0]
hover_data['_y'] = 10*g.xy_nodes.T[1]

In [ ]:
hover_data

In [ ]:
g2 = graphistry.edges(wdf, '_src', '_dst').nodes(hover_data, 'Node').bind(point_x='_x', point_y='_y')
g2.plot()

In [ ]:
g2 = graphistry.edges(edf, 'to_node', 'from_node').nodes(hover_data, 'Node').bind(point_x='_x', point_y='_y')
g2.plot()